In [1]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("pt_core_news_sm")


def remove_stopwords(text):
    doc = nlp(text)
    filtered_words = [token.text for token in doc if not token.is_stop]
    return " ".join(filtered_words)


def similarity_model(user_input):
    data = pd.read_json(
        "/home/maksonvinicio/Documents/GitLab-GitHub/Customer-Care-AI/ml_model/data/data.json"
    ).reset_index(drop=True)

    data_script = pd.read_json(
        "/home/maksonvinicio/Documents/GitLab-GitHub/Customer-Care-AI/ml_model/data/data_script.json"
    ).reset_index(drop=True)

    data["description"] = data["description"].apply(lambda x: x.lower())

    data["description"] = data["description"].apply(remove_stopwords)

    # Instantiate the TF-IDF vectorizer
    vectorizer = TfidfVectorizer(lowercase=True, strip_accents="unicode")

    # Apply TF-IDF on the text dataset
    tfidf_matrix = vectorizer.fit_transform(data["description"])

    user_input = remove_stopwords(user_input)

    # Vetorização do input do usuário
    input_vector = vectorizer.transform([user_input])

    # Cálculo da similaridade de cosseno entre o input do usuário e cada descrição
    similarity_scores = cosine_similarity(input_vector, tfidf_matrix)

    return similarity_scores.argmax()


/home/maksonvinicio/Documents/GitLab-GitHub/Customer-Care-AI/venv/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'pt_core_news_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
from motor.motor_asyncio import AsyncIOMotorClient
import asyncio
from pymongo import MongoClient


DB_HOST_DEV = "mongo"
DB_PASSWORD_DEV = "example"
DB_PORT_DEV = "27017"
DB_DATABASE_DEV = "teste_data"

MONGODB_HOST = "127.0.0.1"  # Ou o endereço IP/nome do servidor MongoDB
MONGODB_PORT = 27017  # Ou a porta correta em que o MongoDB está configurado para ouvir

In [ ]:
DB_URL = f"""mongodb://{MONGODB_HOST}:{DB_PASSWORD_DEV}
@{DB_HOST_DEV}:{DB_PORT_DEV}/"""

# client = AsyncIOMotorClient(DB_URL)

client = MongoClient('localhost', 27017)

database = client[DB_DATABASE_DEV]
conversations_collection = database["bot"]

script_collection = database["script"]

script_collection = database["script"]

# Teste script collection

In [ ]:
script_collection.insert_one(
    {
        "steps": 8,
        "script_id": 4,
        "script": [
            {
                "step": 1,
                "message": "Olá! Bem-vindo ao nosso suporte. Como posso ajudar você hoje?",
            },
            {
                "step": 2,
                "message": "Entendo que você está buscando ajuda para automatizar um processo de atendimento ao cliente no front office. Podemos orientar você nessa questão.",
            },
            {
                "step": 3,
                "message": "Você poderia fornecer mais detalhes sobre o processo que deseja automatizar? Quais são as etapas envolvidas e quais são os principais desafios que você está enfrentando em relação à eficiência?",
            },
            {
                "step": 4,
                "message": "Quais são os objetivos que você deseja alcançar por meio da automação desse processo? Por exemplo, você está buscando reduzir o tempo de resposta, melhorar a qualidade do atendimento ou aumentar a satisfação dos clientes?",
            },
            {
                "step": 5,
                "message": "Com base nas informações fornecidas, vamos fornecer orientações sobre como melhorar a automação desse processo. Após coletarmos todas as informações necessárias, poderemos oferecer soluções mais específicas.",
            },
            {"step": 6, "message": "Coleta das informações"},
            {
                "step": 7,
                "message": "Obrigado por fornecer os detalhes sobre o processo de atendimento ao cliente que você deseja automatizar. Vou encaminhar sua demanda para nossa equipe especializada, que analisará as informações e fornecerá as melhores práticas e soluções adequadas para otimizar a eficiência do seu processo.",
            },
            {
                "step": 8,
                "message": "Se você tiver mais alguma dúvida ou precisar de suporte adicional, não hesite em nos contatar novamente. Estamos aqui para ajudar. Obrigado pelo contato!",
            },
        ],
    }
)

# Teste API

In [ ]:
details = {
	"user_id": 1,
	"message": "Estou com alguns problemas com meu seguro."
}

In [ ]:
# Assuming you have a MongoDB collection named "conversations_collection" and "script_collection"

data = dict(details)

user_id_to_find = data.get("user_id")

# Find the existing document with the specified user_id
if existing_document := conversations_collection.find_one({"user_id": user_id_to_find}):
    # Get the last "step" within the "conversation" list
    last_step = existing_document["conversation"][-1]["step"]
    most_similar_index = existing_document["dialog_id"]

    # Find the corresponding script based on "most_similar_index"
    dados_script = script_collection.find_one({"script_id": int(most_similar_index)})

    # Create the new document for the conversation
    new_document = {
        "step": last_step + 1,
        "user_message": data.get("message"),
        "bot_response": dados_script["script"][last_step]["message"],
    }

    # Append the new document to the existing conversation
    existing_document["conversation"].append(new_document)

    # Update the existing document in the collection with the new conversation
    conversations_collection.update_one(
        {"user_id": user_id_to_find},
        {"$push": {"conversation": new_document}}
    )
else:
    # The user_id doesn't exist yet, so create a new document for the conversation

    # Determine the most similar index based on "data.get("message")"
    most_similar_index = int(similarity_model(data.get("message")))

    # Find the corresponding script based on "most_similar_index"
    dados_script = script_collection.find_one({"script_id": most_similar_index})

    # Get the initial bot response from the script
    bot_response = dados_script["script"][0]["message"]

    # Create the document to insert into the collection
    user_dialog = {
        "dialog_id": most_similar_index,
        "user_id": user_id_to_find,
        "conversation": [
            {
                "step": 1,
                "user_message": data.get("message"),
                "bot_response": bot_response,
            }
        ],
    }
    conversations_collection.insert_one(user_dialog)

    print("Insert")


In [ ]:
# # Provide the connection string to connect to the MongoDB database
# connection_string = DB_URL

# message_history = MongoDBChatMessageHistory(
#     connection_string=connection_string, session_id="test-session"
# )

# message_history.add_user_message("hi!")

# message_history.add_ai_message("whats up?")

In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer
from pymongo import MongoClient
import jwt
import datetime

app = FastAPI()

# Configurando a conexão com o MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]
users_collection = db["users"]

# Chave secreta para assinar os JWTs
SECRET_KEY = "mysecretkey"

# Configurando o esquema de autenticação com OAuth2
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Função para criar um JWT com o ID do usuário
def create_jwt_token(data: dict, expiration: int = 3600):
    to_encode = data.copy()
    expire = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(
        seconds=expiration
    )
    to_encode["exp"] = expire
    return jwt.encode(to_encode, SECRET_KEY, algorithm="HS256")



In [ ]:
# Exemplo de dicionário de dados do usuário
user_data = {
    "user_id": "4",
    "username": "john_doe",
    # ... outros dados do usuário ...
}

# Criar um token JWT com os dados do usuário
token = create_jwt_token(user_data)

print(token)